In [1]:
% matplotlib inline

import numpy as np
import pandas as pd
import json
import sys
import os
import matplotlib
#matplotlib.use('Agg') 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import seaborn as sns
import pdb
import ipdb
import h5py
import importlib
import pickle

# Add root folder to sys path
sys.path.append("../")

# %pylab inline
# inline doesn't give interactive plots
%matplotlib inline 
# %matplotlib notebook
plt.rcParams['figure.figsize'] = (12.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'Blues'

sns.set()

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Import own modules
# Need to use importlib since module starts with a numerical value.

from load_expert_traj import Expert, ExpertHDF5
from grid_world import create_obstacles

plt.rcParams['figure.figsize'] = (6.0, 6.0) # set default size of plots

/home/mohit/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def plot_data_n_colors_policy(data, grid_size, policy,
                              bounds=[-100, -0.5, 0.5, 1000],
                              color_map=['black', 'grey', 'red'],
                              save_path=''):
    
    cmap = matplotlib.colors.ListedColormap(color_map)
    norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(data, cmap=cmap, norm=norm, extent=[0, grid_size, 0, grid_size],
              )

    ax.get_xaxis().set_ticklabels([])
    ax.get_yaxis().set_ticklabels([])
    ax.grid(True)
    ax.autoscale(enable=False)

    for a, m in [(0, '>'), (1, 'v'), (2, '<'), (3, '^')]:
        if len(np.where(policy == a)) > 0:
            x, y = np.where(policy == a)
            # Why this weird math?
            # Original grid map has origin at top left (usual for images).
            # imshow (above) has a property `extent` which allows us to define the origin
            # location when rendering. But ax.scatter (below) doesn't have such an API,
            # it rather defines origin at  bottom left corner hence to convert to top left
            # we do the below math.
            ax.scatter(x=y + 0.5, y=grid_size - 1 - x + 0.5, c='r',
                       s=120, marker=m, edgecolors='none')
    if len(save_path) > 0:
        fig.savefig(save_path)
        
    fig.show()
    
def plot_trajectory(traj_data, grid_size,
                    pred_traj_data=None,
                    obstacles=None,
                    rooms=None,
                    pred_context=[],
                    pred_context_discrete=False,
                    bounds=[-100, -.5, .5, 100],
                    color_map=['black', 'grey', 'red'],
                    save_path='', figsize=(6,6)):
    cmap = matplotlib.colors.ListedColormap(color_map)
    norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)

    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(1, 1, 1)
    img_arr = np.ones(grid_size)
    if obstacles is not None:
        for o in obstacles:
            img_arr[o[1], o[0]] = 0.0
    ax.imshow(img_arr, 
              cmap=cmap,
              norm=norm,
              extent=[0, grid_size[1], 0, grid_size[0]],
              interpolation='none', aspect='equal', origin='lower',
              )

    ax.set_xticks(range(grid_size[1]))
    ax.set_yticks(range(grid_size[0]))
    ax.grid(True)
    
    # Get context colors
    if pred_context_discrete and len(pred_context) > 0:
        num_discrete = len(pred_context[0])
        context_colors = ['g', 'b', 'c', 'm', 'y', 'b']
    else:
        context_colors = ['g']

    for i in range(traj_data.shape[0]):
        x, y = traj_data[i, 0], traj_data[i, 1]
        marker = '.' if i > 0 else '*'
        ax.scatter(x=x + 0.5, y=y + 0.5, c='r',
                       s=120, marker=marker, edgecolors='none')
        
        if pred_traj_data is not None:
            pred_x, pred_y = pred_traj_data[i, 0], pred_traj_data[i, 1]
            pred_color = context_colors[0]
            if len(context_colors) > 1:
                assert np.argmax(pred_context[i]) < len(context_colors)
                pred_color = context_colors[np.argmax(pred_context[i])]
                
            ax.scatter(x=pred_x + 0.5, y=pred_y + 0.75, c=pred_color,
                       s=120, marker=marker, edgecolors='none')

    if len(save_path) > 0:
        fig.savefig(save_path)
    
    fig.tight_layout()
    fig.show()
    
def softmax(x):
    new_x = x - np.max(x,axis=1)[:, np.newaxis]
    denom = np.sum(np.exp(new_x), axis = 1)[:, np.newaxis]
    return np.exp(new_x)/denom

In [ ]:
def get_all_trajectory_from_sequence(states):
    states_int = states.astype(int)

    last_goal_idx = 0
    traj_list = []
    while last_goal_idx < states.shape[0]:
        curr_goal_idx = last_goal_idx
        while curr_goal_idx < states.shape[0]:
            if (states_int[last_goal_idx, 2:].tolist()
                == states_int[curr_goal_idx, 2:].tolist()):
                curr_goal_idx = curr_goal_idx + 1
            else:
                break
        # we have one trajectory
        traj = states[last_goal_idx:curr_goal_idx, :]
        traj_list.append(traj)
        last_goal_idx = curr_goal_idx
        
    return traj_list

def get_multiple_traj_idx_from_goal_list(goal_list):
    last_goal_idx = 0
    goal_idx_list = []
    while last_goal_idx < len(goal_list):
        curr_goal_idx = last_goal_idx
        while curr_goal_idx < len(goal_list):
            if goal_list[last_goal_idx] == goal_list[curr_goal_idx]:
                curr_goal_idx = curr_goal_idx + 1
            else:
                break
        # we have one trajectory
        goal_idx_list.append((last_goal_idx, curr_goal_idx))
        last_goal_idx = curr_goal_idx
        
    return goal_idx_list

def plot_expert_trajectories_from_h5(expert_h5_path,
                                     num_traj_to_plot=10,
                                     obstacles=None,
                                     rooms=None):
    expert = ExpertHDF5(expert_h5_path, 2)
    # Push data into memory
    expert.push()
    
    expert_data = expert.sample(num_traj_to_plot)
    
    i = 0
    while i < num_traj_to_plot:
        ep_states = expert_data.state[i]
        traj_list = get_all_trajectory_from_sequence(ep_states)
        for traj in traj_list:        
            # get (x, y) states for trajectories
            ep_positions = traj[:, :2][::-1]
            # Plot trajectory
            plot_trajectory(ep_positions,
                            (15, 11),
                            color_map=sns.color_palette("Blues_r"),
                            figsize=(6, 6),
                            obstacles=obstacles,
                            rooms=rooms)
            i = i + 1
            if i >= num_traj_to_plot:
                break

obstacles, rooms = create_obstacles(11, 15, env_name='room',
                                    room_size=3)
# plot_expert_trajectories_from_h5('../h5_trajs/room_trajs/',
#                                  num_traj_to_plot=0,
#                                  obstacles=obstacles,
#                                  rooms=rooms,
#                                  )


In [ ]:
results_file = '../results/vae/room_traj/discrete/lr_001_discrete_vae_context_4/results_cp_500.pkl'
discrete_context = True

def plot_pickle_results(results_pkl_path, obstacles, rooms,
                        num_traj_to_plot=10):
    assert os.path.exists(results_pkl_path), \
        'results pickle does not exist {}'.format(results_pkl_path)
    with open(results_pkl_path, 'rb') as results_f:
        results_dict = pickle.load(results_f)
        
        total_traj = len(results_dict['true_traj_state'])
        num_plot_traj = 0
        while num_plot_traj < num_traj_to_plot:
            traj_idx = np.random.randint(total_traj)
            #traj_idx = i
            results_dict['true_goal'][traj_idx]

            goal_idx_list = get_multiple_traj_idx_from_goal_list(
                np.squeeze(results_dict['true_goal'][traj_idx]).tolist())
            
            for goal_start_idx, goal_end_idx in goal_idx_list:
            
                traj_len = goal_end_idx - goal_start_idx
                true_traj, pred_traj = [], []
                pred_context_list = []
                          
                for j in range(traj_len):
                    x_true = (results_dict['true_traj_state'][traj_idx][goal_start_idx + j, 0, :]).tolist()
                    x_pred = (results_dict['pred_traj_state'][traj_idx][goal_start_idx + j, 0, :]).tolist()
                    true_traj.append(x_true)
                    pred_traj.append(x_pred)
                    if results_dict.get('pred_context') is not None:
                        if discrete_context:
                            pred_context = softmax(
                                results_dict['pred_context'][traj_idx][goal_start_idx+j, 0, :]
                            )
                        else:
                            pred_context = results_dict['pred_context'][traj_idx][goal_start_idx+j, 0, :]
                        pred_context_list.append(pred_context)
                
                # Plot trajectory
                plot_trajectory(np.array(true_traj),
                                (15, 11),
                                pred_traj_data=np.array(pred_traj),
                                color_map=sns.color_palette("Blues_r"),
                                figsize=(6, 6),
                                obstacles=obstacles,
                                rooms=rooms,
                                pred_context=pred_context_list,
                                pred_context_discrete=discrete_context,
                               )
                num_plot_traj += 1

plot_pickle_results(results_file, obstacles, rooms, num_traj_to_plot=10)

> <ipython-input-4-c0d640ed75ed>(25)plot_pickle_results()
-> if results_dict.get('pred_context') is not None:
(Pdb) type(results_dict['pred_context'])
<class 'list'>
(Pdb) len(results_dict['pred_context'])
300
(Pdb) results_dict['pred_context'][0]
array([[[-7.91138077e+00, -3.58057251e+01, -9.41675758e+00,
          1.18385906e+01]],

       [[-7.93123341e+00, -3.88793945e+01, -8.70967388e+00,
          9.82479095e+00]],

       [[ 1.06084366e+01, -2.68356781e+01, -3.93933964e+00,
         -5.25124836e+00]],

       [[ 2.94943962e+01,  2.37977695e+01, -2.94667702e+01,
          6.06641054e+00]],

       [[ 3.59078369e+01,  1.35511961e+01, -2.12155399e+01,
         -4.30070448e+00]],

       [[ 3.45404129e+01,  1.09433708e+01, -1.85096531e+01,
         -5.64744377e+00]],

       [[ 2.58786716e+01,  1.16891108e+01, -1.58638144e+01,
         -3.97286725e+00]],

       [[ 1.25883446e+01,  1.42955532e+01, -1.31230659e+01,
          2.23519206e-02]],

       [[ 6.69530153e+00,  2.08424721e+0